<a href="https://colab.research.google.com/github/Docasti/NoCountry/blob/main/intakes_outcomes_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/refugio/aac_intakes_outcomes.csv')

print (f"****Columnas antes de la limpieza y trasformacion de datos****")
print(data.head())

# Contar los valores nulos
print(f"Columnas con datos nulos")
valores_nulos = data.isnull().sum()
print(valores_nulos)

# Manejo de valores nulos
data['outcome_subtype'].fillna('Unknow', inplace=True)
data['outcome_type'].fillna('Unknow', inplace=True)
data['sex_upon_outcome'].fillna('Unknow', inplace=True)
data['sex_upon_intake'].fillna('Unknow', inplace=True)

# Transformar "time_in_shelter"
data['time_in_shelter'] = pd.to_timedelta(data['time_in_shelter'])
# Convertir a dias
data['time_in_shelter_days'] = data['time_in_shelter'].dt.days

# tranf columnas datetime a fecha
data['outcome_date'] = pd.to_datetime(data['outcome_datetime']).dt.date
data['intake_date'] = pd.to_datetime(data['intake_datetime']).dt.date
data['birth_date'] = pd.to_datetime(data['date_of_birth']).dt.date

# Eliminar datetime originales
data.drop(['outcome_datetime', 'intake_datetime', 'date_of_birth'], axis=1, inplace=True)
print(data.head())

# Calcular tiempo de estadia entre outcome_date e intake_date
data['time_in_shelter_days'] = (pd.to_datetime(data['outcome_date']) - pd.to_datetime(data['intake_date'])).dt.days

# Guardar el DataFrame
data.to_csv('/content/aac_intakes_outcomes_1.csv', index=False)

print (f"****Columnas despues de la limpieza y trasformacion de datos****")
# Vistazo rapido de nuestras variable en DF limpio
data_describe = pd.read_csv('/content/aac_intakes_outcomes_1.csv')

# Mostrar un resumen
print(data_describe.describe())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
****Columnas antes de la limpieza y trasformacion de datos****
  age_upon_outcome animal_id_outcome    date_of_birth outcome_subtype  \
0         10 years           A006100  09/07/2007 0:00             NaN   
1          7 years           A006100  09/07/2007 0:00             NaN   
2          6 years           A006100  09/07/2007 0:00             NaN   
3         10 years           A047759  02/04/2004 0:00         Partner   
4         16 years           A134067  16/10/1997 0:00             NaN   

      outcome_type sex_upon_outcome  age_upon_outcome_(days)  \
0  Return to Owner    Neutered Male                     3650   
1  Return to Owner    Neutered Male                     2555   
2  Return to Owner    Neutered Male                     2190   
3         Transfer    Neutered Male                     3650   
4  Return to Owner    Neutered Male              

In [ ]:

# visualizo column relevantes, age_upon_outcome_age_group , tal vez hacer un promedio?
print(data[['animal_type', 'outcome_type', 'time_in_shelter_days', 'age_upon_outcome_age_group']])

      animal_type     outcome_type  time_in_shelter_days  \
0             Dog  Return to Owner                     0   
1             Dog  Return to Owner                     1   
2             Dog  Return to Owner                     1   
3             Dog         Transfer                     4   
4             Dog  Return to Owner                     0   
...           ...              ...                   ...   
79665         Cat         Transfer                     0   
79666       Other       Euthanasia                     0   
79667       Other       Euthanasia                     0   
79668         Dog  Return to Owner                     1   
79669         Dog       Euthanasia                     0   

      age_upon_outcome_age_group  
0                    (7.5, 10.0]  
1                     (5.0, 7.5]  
2                     (5.0, 7.5]  
3                    (7.5, 10.0]  
4                   (15.0, 17.5]  
...                          ...  
79665              (-0.025, 2.5]  

In [ ]:
# visualizo column relevantes, age_upon_outcome_age_group , tal vez hacer un promedio?
print(data[['animal_type', 'outcome_type', 'time_in_shelter_days', 'age_upon_outcome_age_group']])

In [ ]:
# ####### funcion limpiar datos py
def limpiar_datos(data):
    # Contar los valores nulos
    print("Columnas con datos nulos:")
    valores_nulos = data.isnull().sum()
    print(valores_nulos)

    # Manejo de valores nulos
    data['outcome_subtype'].fillna('Unknow', inplace=True)
    data['outcome_type'].fillna('Unknow', inplace=True)
    data['sex_upon_outcome'].fillna('Unknow', inplace=True)
    data['sex_upon_intake'].fillna('Unknow', inplace=True)

    # Transformar "time_in_shelter"
    data['time_in_shelter'] = pd.to_timedelta(data['time_in_shelter'])
    # Convertir a días
    data['time_in_shelter_days'] = data['time_in_shelter'].dt.days

    # Transformar columnas datetime a fecha
    data['outcome_date'] = pd.to_datetime(data['outcome_datetime']).dt.date
    data['intake_date'] = pd.to_datetime(data['intake_datetime']).dt.date
    data['birth_date'] = pd.to_datetime(data['date_of_birth']).dt.date

    # Eliminar datetime originales
    data.drop(['outcome_datetime', 'intake_datetime', 'date_of_birth'], axis=1, inplace=True)

    # Calcular tiempo de estancia entre outcome_date e intake_date
    data['time_in_shelter_days'] = (pd.to_datetime(data['outcome_date']) - pd.to_datetime(data['intake_date'])).dt.days

    return data


In [ ]:
###################funcion limpiar sql

In [ ]:
def clean_and_transform_data(connection):
    cursor = connection.cursor()

    # Actualizar valores nulos en las columnas 'outcome_subtype', 'outcome_type', 'sex_upon_outcome' y 'sex_upon_intake'
    cursor.execute("""
        UPDATE outcome
        SET outcome_subtype = COALESCE(outcome_subtype, 'Unknown'),
            outcome_type = COALESCE(outcome_type, 'Unknown'),
            sex_upon_outcome = COALESCE(sex_upon_outcome, 'Unknown'),
            sex_upon_intake = COALESCE(sex_upon_intake, 'Unknown')
    """)

    # Transformar la columna 'time_in_shelter' a días
    cursor.execute("""
        UPDATE outcome
        SET time_in_shelter_days = EXTRACT(EPOCH FROM time_in_shelter) / 86400, -- Convertir a días
            time_in_shelter = time_in_shelter::interval  -- Transformar a intervalo
    """)

    # Transformar las columnas de fecha y hora a fecha
    cursor.execute("""
        UPDATE outcome
        SET outcome_date = DATE(outcome_datetime),
            intake_date = DATE(intake_datetime),
            birth_date = DATE(date_of_birth)
    """)

    # Eliminar las columnas originales de fecha y hora
    cursor.execute("""
        ALTER TABLE outcome
        DROP COLUMN outcome_datetime,
        DROP COLUMN intake_datetime,
        DROP COLUMN date_of_birth
    """)

    # Calcular el tiempo de estancia entre outcome_date e intake_date
    cursor.execute("""
        UPDATE outcome
        SET time_in_shelter_days = (outcome_date - intake_date)::interval / 86400 -- Convertir a días
    """)

    # Guardar los datos limpios en un nuevo archivo CSV
    cursor.execute("""
        COPY (
            SELECT * FROM outcome
        ) TO '/ruta/de/destino/aac_intakes_outcomes_1.csv' DELIMITER ',' CSV HEADER
    """)

    # Guardar los cambios en la base de datos
    connection.commit()

    cursor.close()

# Llamar a las funciones para establecer la conexión y limpiar los datos
connection = connect_to_database()
clean_and_transform_data(connection)
connection.close()


NameError: name 'connect_to_database' is not defined